# 8팀 (이정수, 이홍규, 정현우) 서울시 따릉이 대여소 주변 환경 기반 자전거 대여 예측

# 데이터 목록

[ 사용 데이터 정리 ]  
1. 서울시 따릉이 대여소별 대여/반납 승객수 정보
    - 제공: 서울 열린데이터 광장
    - 저작권자(제공기관): 서울특별시
    - 데이터 기간: 2022.01. ~ 2022.07.
    - url: https://data.seoul.go.kr/dataList/OA-21229/F/1/datasetView.do  

2. 서울특별시 따릉이대여소 마스터 정보 (대여소 ID 및 위경도 정보)
    - 제공: 서울 열린데이터 광장
    - 저작권자(제공기관): 서울특별시
    - 데이터 기간: 해당사항 없음
    - url: http://data.seoul.go.kr/dataList/OA-21235/S/1/datasetView.do    
  
3. 기상청 > 서울특별시 시간별 날씨 데이터
    - 제공: 기상청 기상자료개방포털
    - 저작권자(제공기관): 기상청
    - 데이터 기간: 2022.01. ~ 2022.07.
    - url: https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36  
    
4. 수치표고모델(DEM)_90M
  - 제공: 국토지리정보원
  - url: http://data.nsdi.go.kr/dataset/20001  

5. 서울시 버스노선 기본정보 항목정보
    - 제공: 서울 열린데이터 광장
    - 저작권자(제공기관): 서울특별시
    - 데이터 기간: 2022.08.23.
    - url: http://data.seoul.go.kr/dataList/OA-15262/F/1/datasetView.do

6. 서울시 버스노선별 정류장별 승하차 인원 정보
    - 제공: 서울 열린데이터 광장
    - 저작권자(제공기관): 서울특별시
    - 데이터 기간: 2022.01. ~ 2022.07.
    - url: http://data.seoul.go.kr/dataList/OA-12912/S/1/datasetView.do

7. 서울시 버스노선별 정류장별 시간대별 승하차 인원 정보
    - 제공: 서울 열린데이터 광장
    - 저작권자(제공기관): 서울특별시
    - 데이터 기간: 2022.01. ~ 2022.07.
    - url: http://data.seoul.go.kr/dataList/OA-12913/S/1/datasetView.do

데이터 저장 경로: "./data/각 데이터폴더명"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import folium
import math

import warnings
warnings.filterwarnings(action="ignore")

# 데이터 처리 및 Merge

### 데이터 로드

- 1. 서울시 따릉이 대여소별 대여/반납 승객수 정보

In [ ]:
# zip 파일 풀기
import zipfile

for i in range(1,8):
    fantasy_zip = zipfile.ZipFile(f'tpss_bcycl_od_statnhm_20220{i}.zip')
    fantasy_zip.extractall('./bicycle_2201_2207')

fantasy_zip.close()

In [ ]:
# 1월 - 7월의 대여 정보 합치기

dir_path = "./bicycle_2201_2207"
file_paths=[]

for (root, directories, files) in os.walk(dir_path):
    for file in files:
        if '.csv' in file:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)

colnames=['기준_날짜', '기준_시간대', '시작_대여소_ID', '종료_대여소_ID', '전체_건수', '전체_이용_분', '전체_이용_거리']

bicycle_df = []
for path in file_paths:
    try:
        bicycle_tmp=pd.read_csv(path,encoding='cp949')
    except UnicodeDecodeError:
        bicycle_tmp=pd.read_csv(path,encoding='utf-8') # 20220330 데이터 unicode 다름
    bicycle_tmp.columns=colnames
    bicycle_df.append(bicycle_tmp)

bicycle_tmp=pd.concat(bicycle_df, axis=0, ignore_index=True)
bicycle_tmp

- 2. 서울특별시 따릉이대여소 마스터 정보

In [ ]:
location=pd.read_csv("./data/서울시-따릉이대여소-마스터-정보.csv",encoding='cp949')
location=location[location['위도']!=0]
location

### 데이터 전처리

- 1. 전처리 _ 서울시 따릉이 대여소별 대여/반납 승객수 정보  

In [ ]:
import datetime

# 전체_이용_분 = 0, 전체_이용_거리 = 0, 시작_대여소=종료_대여소 인 데이터는 제거
bicycle=bicycle_tmp[(bicycle_tmp["전체_이용_분"]>0) & (bicycle_tmp["전체_이용_거리"]>0)]
bicycle=bicycle[bicycle["시작_대여소_ID"]!=bicycle["종료_대여소_ID"]]

# 뒤에서 대여소 위치 데이터와 merge 위해서 형태 변경
bicycle['시작_대여소_ID'] = bicycle['시작_대여소_ID'].str.split('-').str[1]
bicycle['종료_대여소_ID'] = bicycle['종료_대여소_ID'].str.split('-').str[1]
bicycle = bicycle.iloc[:,:5]

# 날짜랑 시간 datetime 형식으로 변경
bicycle['기준_시간대']=bicycle['기준_시간대'].apply(lambda x: "".join([str(x).zfill(4)[:2],'00']))
day_time=bicycle['기준_날짜'].astype(str)+bicycle['기준_시간대']
bicycle['기준_날짜시간']=pd.to_datetime(day_time,format="%Y%m%d%H%M")

# 요일값 추가
days=['월','화','수','목','금','토','일']
bicycle['weekday']=bicycle['기준_날짜시간'].dt.weekday
bicycle['요일']=bicycle.apply(lambda x: days[x['weekday']], axis=1)

bicycle=bicycle.iloc[:,2:]
bicycle.drop('weekday', axis=1, inplace=True)

bicycle.sort_values('기준_날짜시간',inplace=True)
bicycle.reset_index(drop=True,inplace=True)

bicycle['시작_대여소_ID']=bicycle['시작_대여소_ID'].astype(int)
bicycle['종료_대여소_ID']=bicycle['종료_대여소_ID'].astype(int)

bicycle

,시작_대여소_ID,종료_대여소_ID,전체_건수,기준_날짜시간,요일
0,2988,930,1,2022-01-01 00:00:00,토
1,2396,1000,1,2022-01-01 00:00:00,토
2,2761,1005,1,2022-01-01 00:00:00,토
3,2789,2965,1,2022-01-01 00:00:00,토
4,315,2015,1,2022-01-01 00:00:00,토
...,...,...,...,...,...
36066448,2031,2056,1,2022-07-31 23:00:00,일
36066449,717,2380,1,2022-07-31 23:00:00,일
36066450,1868,1582,1,2022-07-31 23:00:00,일
36066451,2037,516,1,2022-07-31 23:00:00,일


- 2. 전처리 _ 서울특별시 따릉이대여소 마스터 정보

In [ ]:
# 해당 부분은 시각화를 위한 df 전처리 코드로 Merge한 최종 df와 관련 없습니다
location=pd.read_csv("서울시 따릉이대여소 마스터 정보.csv",encoding='cp949')
location=location.iloc[:,[0,3,4]]

bicycle_tmp['시작_대여소_ID'] = bicycle_tmp['시작_대여소_ID'].str.split('-').str[1]
bicycle_tmp['종료_대여소_ID'] = bicycle_tmp['종료_대여소_ID'].str.split('-').str[1]
bicycle_tmp = bicycle_tmp.iloc[:,:4]
bicycle_tmp=bicycle_tmp[bicycle_tmp["시작_대여소_ID"]!=bicycle_tmp["종료_대여소_ID"]]

bicycle_tmp['기준_시간대']=bicycle_tmp['기준_시간대'].apply(lambda x: str(x).zfill(4))
day_time=bicycle_tmp['기준_날짜'].astype(str)+bicycle_tmp['기준_시간대']
bicycle_tmp['기준_날짜시간']=pd.to_datetime(day_time,format="%Y%m%d%H%M")
bicycle_tmp=bicycle_tmp.iloc[:,2:]

bicycle_tmp.sort_values('기준_날짜시간',inplace=True)
bicycle_tmp.reset_index(drop=True,inplace=True)

bicycle_tmp['시작_대여소_ID']=bicycle_tmp['시작_대여소_ID'].astype(int)
bicycle_tmp['종료_대여소_ID']=bicycle_tmp['종료_대여소_ID'].astype(int)

bicycle_location=pd.merge(bicycle_tmp, location, how='left', left_on='시작_대여소_ID', right_on='대여소ID')
bicycle_location.rename(columns={'좌표':'시작_좌표'},inplace=True)
bicycle_location=pd.merge(bicycle_location, location, how='left', left_on='종료_대여소_ID', right_on='대여소ID')
bicycle_location.rename(columns={'좌표':'종료_좌표'},inplace=True)
bicycle_location=bicycle_location.iloc[:,[2,4,5,6,8,9,10]]

bicycle_location

-----

### 1. 시간별 대여소별 대여건수 + 날씨 정보(기온, 풍속)

In [ ]:
bicycle_rental=pd.pivot_table(bicycle,index=["기준_날짜시간","요일","시작_대여소_ID"],values="전체_건수",aggfunc='sum')
bicycle_rental.reset_index(inplace=True)
bicycle_rental.rename(columns={'전체_건수':'대여건수'},inplace=True)
bicycle_rental

,기준_날짜시간,요일,시작_대여소_ID,대여건수
0,2022-01-01 00:00:00,토,5,2
1,2022-01-01 00:00:00,토,7,2
2,2022-01-01 00:00:00,토,9,1
3,2022-01-01 00:00:00,토,16,1
4,2022-01-01 00:00:00,토,20,2
...,...,...,...,...
7091696,2022-07-31 23:00:00,일,3047,2
7091697,2022-07-31 23:00:00,일,3051,4
7091698,2022-07-31 23:00:00,일,3060,2
7091699,2022-07-31 23:00:00,일,3094,2


In [ ]:
temperature=pd.read_csv("OBS_ASOS_TIM_20221013044255.csv",encoding='cp949')
temperature['기준_날짜시간']=pd.to_datetime(temperature['일시'],format="%Y-%m-%d %H:%M")
temperature=temperature.iloc[:,3:]
temperature.columns=['기온','풍속','습도','기준_날짜시간']
temperature

,기온,풍속,습도,기준_날짜시간
0,-8.5,1.9,41,2022-01-01 00:00:00
1,-9.2,1.8,42,2022-01-01 01:00:00
2,-9.5,1.2,43,2022-01-01 02:00:00
3,-9.3,1.4,46,2022-01-01 03:00:00
4,-9.6,1.7,48,2022-01-01 04:00:00
...,...,...,...,...
5083,26.7,2.3,92,2022-07-31 19:00:00
5084,26.4,2.8,93,2022-07-31 20:00:00
5085,26.0,2.3,94,2022-07-31 21:00:00
5086,25.7,2.3,95,2022-07-31 22:00:00


In [ ]:
bicycle_temperature=pd.merge(bicycle_rental, temperature, on='기준_날짜시간')
bicycle_temperature

,기준_날짜시간,요일,시작_대여소_ID,대여건수,기온,풍속,습도
0,2022-01-01 00:00:00,토,5,2,-8.5,1.9,41
1,2022-01-01 00:00:00,토,7,2,-8.5,1.9,41
2,2022-01-01 00:00:00,토,9,1,-8.5,1.9,41
3,2022-01-01 00:00:00,토,16,1,-8.5,1.9,41
4,2022-01-01 00:00:00,토,20,2,-8.5,1.9,41
...,...,...,...,...,...,...,...
7091696,2022-07-31 23:00:00,일,3047,2,25.5,2.5,95
7091697,2022-07-31 23:00:00,일,3051,4,25.5,2.5,95
7091698,2022-07-31 23:00:00,일,3060,2,25.5,2.5,95
7091699,2022-07-31 23:00:00,일,3094,2,25.5,2.5,95


In [ ]:
bicycle_temperature.to_csv('bicycle_temperature.csv', encoding='cp949')

----------

### 2. 고도, 주변 정류장 수 Merge

In [ ]:
try:
    from haversine import haversine_vector
except:
    !pip install haversine
    from haversine import haversine_vector

try:
    import geopandas as gpd
except :
    !pip install geopandas
    import geopandas as gpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 11.9 MB/s 
     |████████████████████████████████| 16.7 MB 491 kB/s 
     |████████████████████████████████| 6.3 MB 60.3 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

def add_num_bus(df: pd.DataFrame) -> pd.DataFrame:
    bus_stop = pd.read_csv("./data/BUS_STATION_BOARDING_MONTH_202209.csv", encoding="cp949")
    bus_stop = bus_stop[bus_stop["표준버스정류장ID"] <= 124900137]
    bus_stop = bus_stop.sort_values("표준버스정류장ID")
    bus_stop = (
        bus_stop[["표준버스정류장ID", "노선명"]]
        .groupby(["표준버스정류장ID", "노선명"])
        .count()
        .reset_index()
    )
    bus_stop["const"] = 1
    bus_stop = bus_stop.groupby("표준버스정류장ID").sum().reset_index()
    bus_stop = bus_stop.rename(columns={"표준버스정류장ID": "인근정거장", "const": "다니는버스수"})
    df["인근정거장"] = df["인근정거장"].fillna(-1).astype(int)
    return df.merge(bus_stop, how="left", left_on="인근정거장", right_on="인근정거장")


def find_nearest(
    df: pd.DataFrame, lat: np.ndarray, lng: np.ndarray, target: np.ndarray
):
    assert lat.ndim == lng.ndim == target.ndim == 1
    assert lat.shape[0] == lng.shape[0] == target.shape[0]

    tgt: np.ndarray = np.vstack([lat, lng]).T
    assert tgt.shape[0] == lat.shape[0]

    answer = []
    for i, row in df.iterrows():
        yx = row[["위도", "경도"]].values.reshape(1, 2).astype(float)
        yx = np.repeat(yx, len(tgt), axis=0)
        answer.append(haversine_vector(yx, tgt))
    answer
    idx = np.argmin(np.vstack(answer), axis=-1)
    return target[idx]


def add_dem_feature(df: pd.DataFrame):
    dem = []
    for dem_path in Path("./dataDEM/서울특별시/2014 서울특별시[ascii]").glob("*.txt"):
        dem.append(pd.read_csv(dem_path, sep=" ", header=None))
    dem = pd.concat(dem).reset_index(drop=True)
    dem.columns = ["x", "y", "z"]
    dem = gpd.GeoDataFrame(
        data=dem.z, geometry=gpd.points_from_xy(x=dem.x, y=dem.y), crs="EPSG:5186",
    )
    dem = dem.to_crs("EPSG:4326")

    x_std, y_std = df["경도"].std(), df["위도"].std()
    x_candidates: np.ndarray = dem.geometry.x.values
    y_candidates: np.ndarray = dem.geometry.y.values

    df_chunks = []
    for i in range(0, 3500, 500,):
        df_chunk = df[i : i + 500].reset_index(drop=True)
        x = df_chunk["경도"].values.reshape(-1, 1)
        y = df_chunk["위도"].values.reshape(-1, 1)
        indices = np.argmin(
            np.abs(x - x_candidates) / x_std + np.abs(y - y_candidates) / y_std, axis=-1
        )
        df_chunk["dem"] = dem.loc[indices, "z"].values
        df_chunks.append(df_chunk)

    return pd.concat(df_chunks)

----------

In [ ]:
from typing import Callable
def iief(func: Callable):
    """전역 네임스페이스 오염을 막기 위해 IIEF를 선언했습니다.
    데코레이터처럼 사용하려고 합니다."""

    func()

In [ ]:
@iief
def _():
    """ 고도와 주변 정류장 수를 Merge하고 완료된 파일을
    `./merged_data/서울시_따릉이대여소별_고도_지하철역_정거장매핑.csv`에 저장합니다.   """
    df = pd.read_csv("./data/서울시-따릉이대여소-마스터-정보.csv",encoding='cp949')
    df = add_dem_feature(df)

    N: int = len(df)

    temp = df[df["위도"] == 0.0]
    df = df[df["위도"] > 0.0]

    target_df = pd.read_csv(
        "./data/서울교통공사_1_8호선 역사 좌표(위경도) 정보_20211231.csv", encoding="cp949"
    )
    df["인근역"] = find_nearest(
        df, target_df["위도"].values, target_df["경도"].values, target_df["역명"].values
    )

    target_df = pd.read_csv(
        "./data/서울시버스정류소좌표데이터(2022.08.24).csv", encoding="cp949"
    )

    df["인근정거장"] = find_nearest(
        df,
        target_df["좌표Y"].values,
        target_df["좌표X"].values,
        target_df["NODE_ID"].values,
    )
    df = pd.concat([temp, df]).sort_index()

    assert N == len(df)
    df = add_num_bus(df)

    df.to_csv("./merged_data/서울시_따릉이대여소별_고도_지하철역_정거장매핑.csv", index=False)


### 3. 주변 대중교통 이용자수 Merge

자전거 대여를 기준으로 보았을 때, 대중교통 승차 대신 하차가 직접적인 관여를 하기 때문에, 하차 승객 수에 대한 데이터를 추출한다.

In [ ]:
CONST_DIR_BUSDATA_TIME = "./data/서울시 버스노선별 정류장별 시간대별 승하차 인원 정보"
CONST_DIR_BUSDATA_MONTH = "./data/서울시 버스노선별 정류장별 승하차 인원 정보"
CONST_DIR_MERGEDATA = "./merged_data"

CONST_LIST_COLUMNS = ['00시하차총승객수', '1시하차총승객수', '2시하차총승객수', '3시하차총승객수', '4시하차총승객수', '5시하차총승객수', '6시하차총승객수', '7시하차총승객수', '8시하차총승객수', '9시하차총승객수', '10시하차총승객수', '11시하차총승객수', '12시하차총승객수', '13시하차총승객수', '14시하차총승객수', '15시하차총승객수', '16시하차총승객수', '17시하차총승객수', '18시하차총승객수', '19시하차총승객수', '20시하차총승객수', '21시하차총승객수', '22시하차총승객수', '23시하차총승객수']

In [ ]:
time_file_list = os.listdir(CONST_DIR_BUSDATA_TIME)
time_file_list.remove(".DS_Store")
time_file_list.sort()
time_file_list

month_file_list = os.listdir(CONST_DIR_BUSDATA_MONTH)
month_file_list.remove(".DS_Store")
month_file_list.sort()
month_file_list

In [ ]:
def merged_bus_df(path1, path2, bus_id_list):
    time_data_path = os.path.join(CONST_DIR_BUSDATA_TIME, path1)
    time_df = pd.read_csv(time_data_path, encoding="cp949")
    # 대여소별 대여 개수를 예측하는 것이기 때문에 승차 승객이 아닌 하차 승객을 비교 대상으로 선정한다.
    col_list = list(time_df.columns)
    for col in col_list:
        if "승차" in col:
            col_list.remove(col)

    col_list.remove("등록일자")
    # print("time columns: ", col_list)
    time_df = time_df[col_list]
    # time_df = time_df.groupby(["사용년월", "표준버스정류장ID", "버스정류장ARS번호", "역명"]).sum().reset_index(drop=False)
    time_df['total'] = time_df[CONST_LIST_COLUMNS].sum(axis=1)
    for col in CONST_LIST_COLUMNS:
        time_df[col] = time_df[col]/time_df['total']

    time_df.fillna(0,inplace=True)
    time_df['표준버스정류장ID'] = time_df['표준버스정류장ID'].astype(int)
    time_df["버스정류장ARS번호"] = time_df["버스정류장ARS번호"].astype(str)
    for i in range(len(bus_id_list)):
        df1 = time_df[time_df['표준버스정류장ID']== bus_id_list[i]].reset_index(drop=True)
        if i ==0:
            bus_time_df = df1
        else:
            bus_time_df = pd.concat((bus_time_df, df1), ignore_index=True)

    month_data_path = os.path.join(CONST_DIR_BUSDATA_MONTH, path2)
    month_df = pd.read_csv(month_data_path, encoding="cp949")
    month_df["버스정류장ARS번호"] = month_df["버스정류장ARS번호"].astype(str)
    # print("month columns: ", list(month_df.columns))

    if '표준버스정류장ID' in month_df.columns:
        bus_df = pd.merge(month_df, bus_time_df, how='inner', left_on=['노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명'], right_on=['노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명'])
    else:
        bus_df = pd.merge(month_df, bus_time_df, how='inner', left_on=['노선번호', '노선명', '버스정류장ARS번호', '역명'], right_on=['노선번호', '노선명', '버스정류장ARS번호', '역명'])
    for col in CONST_LIST_COLUMNS:
            bus_df[col] = bus_df['하차총승객수'] * bus_df[col]
            bus_df[col] = bus_df[col].round().astype(int)
    bus_df['total'] = bus_df[CONST_LIST_COLUMNS].sum(axis=1)
    # print(bus_df.columns)
    if "표준버스정류장ID" in bus_df.columns:
        bus_df = bus_df.groupby(["사용일자", "표준버스정류장ID", "버스정류장ARS번호", "역명"]).sum()[CONST_LIST_COLUMNS].reset_index(drop=False)
    elif "표준버스정류장ID_y" in bus_df.columns:
        bus_df = bus_df.groupby(["사용일자", "표준버스정류장ID", "버스정류장ARS번호", "역명"]).sum()[CONST_LIST_COLUMNS].reset_index(drop=False)
    else:
        raise ValueError(f"Can't fine '표준버스정류장ID' column in 'bus_df' columns:{list(bus_df.columns)}")
    return bus_df

In [ ]:
# 대여소 인근 버스정류장 ID 추출
CONST_PATH_DATA  = "./merged_data/서울시_따릉이대여소별_고도_지하철역_정거장매핑.csv"
df = pd.read_csv(CONST_PATH_DATA)
df = df.drop(df[df['위도']==0].index).reset_index(drop=True)
df.isnull().sum()
bus_id_list = list(df['인근정거장'].unique())

In [ ]:
for i in range(7):
    path1 = time_file_list[i]
    path2 = month_file_list[i]
    print(f"start to files {path1}, {path2}")
    df = merged_bus_df(path1, path2, bus_id_list)
    if i == 0:
        total_bus_df = df.copy()
    else:
        total_bus_df = pd.concat((total_bus_df, df), ignore_index=True)

In [ ]:
df = total_bus_df[CONST_LIST_COLUMNS]
df = df.transpose().reset_index(drop=False)
df['index'] = df['index'].str[:-7].astype(int)
df.set_index('index', inplace=True)

In [ ]:
for i in range(len(df.columns)):
    print(f"processing...{i+1}/{len(df.columns)}")
    col = list(df.columns)[i]
    df1 = df[col].reset_index(drop=False)
    df1.columns = ["시간", "버스하차승객수"]
    for j in ["사용일자", "표준버스정류장ID", "버스정류장ARS번호", "역명"]:
        df1[j] = ""
        df1[j] = total_bus_df[j][col]
    if i==0:
        bus_df = df1
    else:
        bus_df = pd.concat((bus_df,df1), ignore_index=True)
    del df1

In [ ]:
bus_df.to_csv("./merged_data/bus.csv", encoding="utf-8", index=False)

## 데이터 결합

In [ ]:
dem_path = os.path.join(CONST_DIR_MERGEDATA, "서울시_따릉이대여소별_고도_지하철역_정거장매핑.csv")
bus_path = os.path.join(CONST_DIR_MERGEDATA, "bus.csv")

dem_df = pd.read_csv(dem_path, encoding='utf8')
bus_df = pd.read_csv(bus_path, encoding='utf8')

In [ ]:
bicycle_temperature['대여소_ID'] = "ST-" + bicycle_temperature['시작_대여소_ID'].astype(str)
bicycle_temperature

In [ ]:
df = pd.merge(bicycle_temperature, dem_df, how='left', on='대여소_ID')
df = df[['기준_날짜시간', '요일', '대여건수', '기온', '풍속', '습도', '대여소_ID', 'dem', '인근역', '인근정거장', '다니는버스수']]
df["사용일자"] = df["기준_날짜시간"].astype(str).str.split(" ").str[0].str.replace('-', '').astype(int)
df["시간"] = df["기준_날짜시간"].astype(str).str.split(" ").str[1].str[:2].astype(int)
bus_df = bus_df[['사용일자','시간', '표준버스정류장ID', '버스하차승객수']]
df1 = pd.merge(df, bus_df, how = 'left', left_on=['사용일자', '시간', '인근정거장'], right_on=['사용일자', '시간', '표준버스정류장ID'])

In [ ]:
df1.to_csv("./merged_data/merged_data1.csv", encoding='utf8', index=False)

# 데이터 EDA 및 시각화

- 따릉이 이용시간별 대여수 확인

In [ ]:
bicycle_viz1=bicycle_tmp.copy()
bicycle_viz1['기준_시간대']=bicycle_viz1['기준_시간대'].apply(lambda x: "".join([str(x).zfill(4)[:2],'00']))
bicycle_viz1=pd.pivot_table(bicycle_viz1,index="기준_시간대",values="전체_건수",aggfunc='sum')
bicycle_viz1.reset_index(inplace=True)

plt.figure(figsize=(15,5))
plt.bar(bicycle_viz1['기준_시간대'],bicycle_viz1['전체_건수'],color='plum')
plt.show()

- 따릉이 대여/반환 지역 연결 지도 시각화

In [ ]:
bicycle_viz2=bicycle_location[["위도_x","경도_x","위도_y","경도_y","전체_건수"]]
bicycle_viz2=pd.pivot_table(bicycle_viz2,index=["위도_x","경도_x","위도_y","경도_y"],values="전체_건수",aggfunc='sum')
bicycle_viz2.reset_index(inplace=True)
bicycle_viz2.rename(columns={'기준_날짜시간':'횟수'},inplace=True)

# 대여-반환 경로 이용자수 4분위 수 구하기
root_cnt=list(np.percentile(bicycle_viz2['횟수'].sort_values().unique(),[25,50,75],interpolation='nearest'))

# 대여-반환 경로 이용자 수 별 df 분할
bicycle_viz2_25=bicycle_viz2[bicycle_viz2['횟수']<root_cnt[0]]
bicycle_viz2_50=bicycle_viz2[(bicycle_viz2['횟수']>=root_cnt[0]) & (bicycle_viz2['횟수']<root_cnt[1])]
bicycle_viz2_75=bicycle_viz2[(bicycle_viz2['횟수']>=root_cnt[1]) & (bicycle_viz2['횟수']<root_cnt[2])]
bicycle_viz2_100=bicycle_viz2[bicycle_viz2['횟수']>=root_cnt[2]]

In [ ]:
center = [37.541, 126.986]

'''
mapp = folium.Map(location=center, zoom_start=11)

folium.PolyLine(
    locations = [list(zip(bicycle_viz2_25['위도_x'],bicycle_viz2_25['경도_x'])),list(zip(bicycle_viz2_25['위도_y'],bicycle_viz2_25['경도_y']))],
    opacity = 0.3 , color='red'
).add_to(mapp)
folium.PolyLine(
    locations = [list(zip(bicycle_viz2_50['위도_x'],bicycle_viz2_50['경도_x'])),list(zip(bicycle_viz2_50['위도_y'],bicycle_viz2_50['경도_y']))],
    opacity = 0.3 , color='green'
).add_to(mapp)
folium.PolyLine(
    locations = [list(zip(bicycle_viz2_75['위도_x'],bicycle_viz2_75['경도_x'])),list(zip(bicycle_viz2_75['위도_y'],bicycle_viz2_75['경도_y']))],
    opacity = 0.3 , color='blue'
).add_to(mapp)
folium.PolyLine(
    locations = [list(zip(bicycle_viz2_100['위도_x'],bicycle_viz2_100['경도_x'])),list(zip(bicycle_viz2_100['위도_y'],bicycle_viz2_100['경도_y']))],
    opacity = 0.3 , color='purple'
).add_to(mapp)
mapp
'''

![map movement](https://user-images.githubusercontent.com/47911773/194584414-075d4c51-a782-45fd-b297-787a17387825.png)

- 주말 위치별 대여수 확인 (데이터 크기가 너무 커서 이미지로 대체)

In [ ]:
bicycle_rental_wnd=bicycle_rental[(bicycle_rental['요일']=="토") or (bicycle_rental['요일']=="일")]

In [ ]:
import folium
from folium.plugins import MarkerCluster

'''
m = folium.Map(
    location=[37.541, 126.986],
    zoom_start=15
)

coords = bicycle_rental_wnd[['시작_위도', '시작_경도']]


marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(coords['시작_위도'], coords['시작_경도']):
    folium.Marker([lat, long], icon = folium.Icon(color="green")).add_to(marker_cluster)
m
'''

![map cluster](https://user-images.githubusercontent.com/47911773/194583610-c7456a02-a125-4172-b46c-68f8e89d3433.png)

## 고도(DEM) 피쳐에 대한 F-Test 및 시각화

In [ ]:
import numpy as np
from scipy import stats

#define F-test function
def f_test(df: pd.DataFrame):
    """__Summary__

    * `df`의 `df.dem`을 기준으로 상위 50%, 하위 50% 집단을 분리합니다.
    * 그리고 "대여수" 피쳐와 "반납수" 피쳐의 f-test 수행하고 그 결과를 데이터프레임으로 반환합니다.
    * 전역 네임스페이스를 오염시키지 않고자 함수로 묶었습니다.
    * 인자로 전달된 `df: DataFrame`인자에는 `"대여수", "반납수", "dem"` 컬럼이 포함되어 있어야 합니다.  """

    for col in  ["대여수", "반납수", "dem"] :
        assert col in df.columns


    df_sorted = df.sort_values("dem")
    __N: int = len(df_sorted)//2
    data = []
    index = ["대여수", "반납수"]
    for col in index:
        temp = df_sorted[col].values
        x, y = temp[__N:2*__N], temp[:__N]
        f = np.var(x, ddof=1)/np.var(y, ddof=1)
        dfn = x.size-1
        dfd = y.size-1
        p = 1 - stats.f.cdf(f, dfn, dfd)
        data.append([f, p])

    return pd.DataFrame(
        data=data,
        index=index,
        columns=["f-stats", "p-value"]
    )

@iief
def _():
    pd.read_csv("./merged_data/서울시_따릉이대여소별_고도_지하철역_정거장매핑.csv")
    f_test()

In [ ]:

def plot_dem(df: pd.DataFrame, ax=None, **kwargs) -> None:
    """__Summary__

    * `df`의 `df.dem`을 X축으로 `반납수/대여수`를 y축으로 scatter plot을 그립니다.
    * 인자로 전달된 `df: DataFrame`인자에는 `"대여수", "반납수", "dem"` 컬럼이 포함되어 있어야 합니다.  """

    for col in  ["대여수", "반납수", "dem"] :
        assert col in df.columns

    if ax is None:
        fig, ax = plt.subplots(figsize=(10,5))

    ax.scatter(df["dem"], df["반납수"]/df["대여수"], **kwargs)




# 머신러닝학습 및 성능 평가

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

random.seed(42)

In [ ]:
# 데이터 로드
mergedata = pd.read_csv('merged_data1.csv')
mergedata.drop(['기준_날짜시간','인근역','인근정거장','사용일자','표준버스정류장ID'], axis=1, inplace=True)
mergedata.set_index('대여소_ID',inplace=True)
mergedata

### Merge 데이터 전처리

In [ ]:
# 머신러닝 학습 가능한 형태로 변경
def time_category(time):
    if time < 3: return 3
    elif time < 6: return 6
    elif time < 9: return 9
    elif time < 12: return 12
    elif time < 15: return 15
    elif time < 18: return 18
    elif time < 21: return 21
    else: return 24

mergedata['다니는버스수']=mergedata['다니는버스수'].astype(int)
mergedata['시간']=mergedata['시간'].apply(lambda x: time_category(x))
mergedata['버스하차승객수']=mergedata['버스하차승객수'].astype(int)

mergedata=pd.get_dummies(mergedata, columns=['요일','시간'])
mergedata

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(df.drop('대여건수',axis=1),
                                                    df[['대여건수']], # 이부분은 y로 pd.Series가 아니라 pd.DataFrame을 넣어야하는 경우가 종종 있었던 거 같아 수정했습니다. (수정자: 이홍규)
                                                    test_size=0.2,
                                                    random_state=42)
# X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42) # 아래 grid search 에서 cross validation 진행하므로 split 필요 없어서 수정. (수정자: 이정수)

### 머신러닝 학습

대여소별 자전거 대여 수를 예측하기 위해 SVM, RF, XGBoost 모델을 기반으로 예측모델을 학습 및 평가한다.

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [ ]:
# 데이터 학습 - SVR
pipeline = Pipeline([('Scaler', StandardScaler()), ('SVR', SVR())]) # scalar 적용

kernel_list = ['linear','rbf']
C_list = [0.1, 1, 10]
## parameter 키는 pipeline.get_params().keys()를 통해 확인가능
parameters = {'SVR__kernel':kernel_list, 'SVR__C':C_list}

svm_grid = GridSearchCV(pipeline, parameters, cv=5, scoring = 'neg_root_mean_squared_error') # 5-fold cross validation
svm_grid.fit(X_train, y_train)

# 최적 파라미터로 최종모형 생성 및 테스트셋 성능평가
reg = SVR(C = svm_grid.best_params_['SVR__C'],
          kernel = svm_grid.best_params_['SVR__kernel'])
reg.fit(X_train, y_train)

y_pred=reg.predict(X_test)
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("SVR RMSE':{}".format(RMSE))
print("SVR best C':{}, kernel':{}".format(svm_grid.best_params_['SVR__C'],svm_grid.best_params_['SVR__kernel']))

In [ ]:
# 데이터 학습 - RF
pipeline = Pipeline([('Scaler', StandardScaler()), ('RF', RandomForestRegressor())]) # scalar 적용

max_depth_list = [5,10,15]
min_samples_leaf_list = [5,15]
n_estimators_list= [50,100,200]

parameters = {'RF__max_depth':max_depth_list,
              'RF__min_samples_leaf':min_samples_leaf_list,
              'RF__n_estimators':n_estimators_list}

rf_grid = GridSearchCV(pipeline, parameters, cv=5, scoring = 'neg_root_mean_squared_error') # 5-fold cross validation
rf_grid.fit(X_train, y_train)

# 최적 파라미터로 최종모형 생성 및 테스트셋 성능평가
## parameter 키는 pipeline.get_params().keys()를 통해 확인가능
reg = RandomForestRegressor(max_depth = rf_grid.best_params_['RF__max_depth'],
                            min_samples_leaf = rf_grid.best_params_['RF__min_samples_leaf'],
                            n_estimators = rf_grid.best_params_['RF__n_estimators'])
reg.fit(X_train, y_train)

y_pred=reg.predict(X_test)
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("RF RMSE':{}".format(RMSE))
print("RF best max_depth':{}, min_samples_leaf':{}, n_estimators':{}".format(rf_grid.best_params_['RF__max_depth'],
                                                                             rf_grid.best_params_['RF__min_samples_leaf'],
                                                                             rf_grid.best_params_['RF__n_estimators']))

In [ ]:
# 데이터 학습 - XGBoosting
pipeline = Pipeline([('Scaler', StandardScaler()), ('XGB', XGBRegressor())]) # scalar 적용

lambda_list = [1, 2, 3]
gamma_list = [10, 50, 100]
parameters = {'XGB__reg_lambda':lambda_list, 'XGB__gamma':gamma_list}

xgb_grid = GridSearchCV(pipeline, parameters, cv=5, scoring = 'neg_root_mean_squared_error') # 5-fold cross validation
xgb_grid.fit(X_train, y_train)

# 최적 파라미터로 최종모형 생성 및 테스트셋 성능평가
## parameter 키는 pipeline.get_params().keys()를 통해 확인가능
reg = XGBRegressor(reg_lambda = xgb_grid.best_params_['XGB__reg_lambda'],
                   gamma = xgb_grid.best_params_['XGB__gamma'])
reg.fit(X_train, y_train)

y_pred=reg.predict(X_test)
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("XGB RMSE':{}".format(RMSE))
print("XGB best reg_lambda':{}, gamma':{}".format(xgb_grid.best_params_['XGB__reg_lambda'],xgb_grid.best_params_['XGB__gamma']))